In [30]:
import pandas as pd
import numpy as np
import seaborn as sb
import numba
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

df = pd.read_excel('energydata_complete.xlsx')
column_names = {'T1': 'Temperature in kitchen area, in Celsius', 
                'RH_1' : 'Humidity in kitchen area, in %', 'T2': 'Temperature in living room area, in Celsius', 
                'RH_2': 'Humidity in living room area, in %','T3': 'Temperature in laundry room area',
                'RH_3': 'Humidity in laundry room area, in %','T4': 'Temperature in office room, in Celsius',                
                'RH_4': 'Humidity in office room, in %', 'T5': 'Temperature in bathroom, in Celsius', 
                'RH_5': 'Humidity in bathroom, in %','T6': 'Temperature outside the building (north side), in Celsius', 
                'RH_6': 'Humidity outside the building (north side), in %', 'T7': 'Temperature in ironing room , in Celsius', 
                'RH_7': 'Humidity in ironing room, in %', 'T8': 'Temperature in teenager room 2, in Celsius',
                'RH_8': 'Humidity in teenager room 2, in %', 'T9': 'Temperature in parents room, in Celsius', 
                'RH_9': 'Humidity in parents room, in %', 
                'T_out': 'Temperature outside (from Chievres weather station), in Celsius', 
                'Press_mm_hg': 'Pressure (from Chievres weather station), in mm Hg',
                'RH_out': 'Humidity outside (from Chievres weather station), in %', 
                'Windspeed': 'Wind speed (from Chievres weather station), in m/s',
                'Visibility': 'Visibility (from Chievres weather station), in km', 
                'Tdewpoint': 'Tdewpoint (from Chievres weather station), Â°C','rv1': 'Random variable 1', 
                'rv2': 'Random variable 2'}
df = df.rename(columns = column_names)

df = df.drop(["date", "lights"], axis = 1)
Appliance_target = df.drop("Appliances", axis = 1)
Appliance = df["Appliances"]
x_train, x_test, y_train, y_test = train_test_split(Appliance_target,Appliance,test_size = 0.3, random_state = 42)

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

mm = LinearRegression()
mm.fit(x_train_scaled, y_train)
y_pred = mm.predict(x_test_scaled)


mae = mean_absolute_error(y_test, y_pred)
print(round(mae, 2))

mse = np.sqrt(mean_squared_error(y_test, y_pred))
print(round(mse, 2))

print(sum((y_test - y_pred)**2))

print(r2_score(y_test, y_pred))

ridge = Ridge(alpha = 0.4)
ridge.fit(x_train, y_train)

y_predr = ridge.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_predr))
print(round(rmse,2))

lasso = Lasso(alpha = 0.001)
lasso.fit(x_train, y_train)

y_predl = lasso.predict(x_test)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_predl))
print(round(rmse, 3))

def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(mm, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso, x_train, 'Lasso_Weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on = 'Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
print(final_weights)

49.09
97.53
56322202.36591644
0.07671279829086108
93.64
93.64
                                             Features  Linear_Model_Weight  \
0                  Humidity in living room area, in %          -469.521362   
1   Temperature outside (from Chievres weather sta...          -344.389845   
2         Temperature in living room area, in Celsius          -252.710373   
3             Temperature in parents room, in Celsius          -203.236627   
4                   Humidity in teenager room 2, in %          -168.627161   
5   Humidity outside (from Chievres weather statio...           -83.107599   
6                      Humidity in ironing room, in %           -47.563552   
7                      Humidity in parents room, in %           -42.586339   
8                 Temperature in bathroom, in Celsius           -16.752822   
9             Temperature in kitchen area, in Celsius            -3.510725   
10                                  Random variable 1             0.823870   
11

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.136e+06, tolerance: 1.464e+04
  model = cd_fast.enet_coordinate_descent(
